In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import tldextract
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

# ✅ **Load Dataset**
df = pd.read_csv("phisher.csv")  # Change to your dataset path

# ✅ **Convert Labels to Numeric**
df["status"] = df["Label"].replace({"Legitimate": 0, "Phishing": 1})

# ✅ **Feature Engineering Using `tldextract`**
def extract_features(url):
    extracted = tldextract.extract(url)
    features = {
        "domain": extracted.domain if extracted.domain else "unknown",
        "suffix": extracted.suffix if extracted.suffix else "unknown",
        "subdomain": extracted.subdomain if extracted.subdomain else "unknown",
        "url_length": len(url),
        "dot_count": url.count('.'),
        "hyphen_count": url.count('-'),
        "slash_count": url.count('/'),
        "digit_count": sum(c.isdigit() for c in url),
        "is_https": 1 if url.startswith("https") else 0
    }
    return features

df_features = df["URL"].apply(extract_features).apply(pd.Series)

# ✅ **Encode Categorical Features**
le_domain = LabelEncoder()
df_features["domain"] = le_domain.fit_transform(df_features["domain"])

le_suffix = LabelEncoder()
df_features["suffix"] = le_suffix.fit_transform(df_features["suffix"])

le_subdomain = LabelEncoder()
df_features["subdomain"] = le_subdomain.fit_transform(df_features["subdomain"])

# ✅ **Prepare Training Data**
X = df_features  # Use extracted features
y = df["status"]

# ✅ **Train-Test Split**
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ✅ **Feature Scaling**
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ✅ **Train LightGBM Model**
lgb_model = lgb.LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)
lgb_model.fit(X_train_scaled, y_train)

# ✅ **Train CatBoost Model**
cat_model = CatBoostClassifier(
    iterations=300,
    depth=5,
    learning_rate=0.02,
    l2_leaf_reg=8,
    random_strength=5,
    loss_function="Logloss",
    eval_metric="AUC",
    early_stopping_rounds=20,
    verbose=50,
    random_seed=42
)
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

# ✅ **Evaluate Models**
lgb_acc = accuracy_score(y_test, lgb_model.predict(X_test_scaled))
cat_acc = accuracy_score(y_test, cat_model.predict(X_test))

print(f"\n✅ LightGBM Test Accuracy: {lgb_acc:.4f}")
print(f"✅ CatBoost Test Accuracy: {cat_acc:.4f}")

# 🚀 **URL Prediction Function (Handles Unseen Domains)**
def predict_url(url):
    extracted_features = extract_features(url)

    # Handle unseen labels by assigning them a default category
    extracted_features["domain"] = le_domain.transform([extracted_features["domain"]])[0] if extracted_features["domain"] in le_domain.classes_ else -1
    extracted_features["suffix"] = le_suffix.transform([extracted_features["suffix"]])[0] if extracted_features["suffix"] in le_suffix.classes_ else -1
    extracted_features["subdomain"] = le_subdomain.transform([extracted_features["subdomain"]])[0] if extracted_features["subdomain"] in le_subdomain.classes_ else -1

    feature_vector = np.array(list(extracted_features.values())).reshape(1, -1)
    feature_vector_scaled = scaler.transform(feature_vector)

    lgb_pred = lgb_model.predict_proba(feature_vector_scaled)[0][1]
    cat_pred = cat_model.predict_proba(feature_vector)[0][1]

    lgb_result = "Phishing" if lgb_pred > 0.5 else "Legitimate"
    cat_result = "Phishing" if cat_pred > 0.5 else "Legitimate"

    print(f"\n🔍 URL: {url}")
    print(f"🚀 LightGBM Prediction: {lgb_result} (Confidence: {lgb_pred:.4f})")
    print(f"🚀 CatBoost Prediction: {cat_result} (Confidence: {cat_pred:.4f})")

# ✅ **Test Predictions**
test_urls = [
    "http://paypal-secure-login.com",
    "https://www.google.com",
    "https://www.amazon.com",
    "https://careeracademyschool.com",
    "http://start-trezoroi.webflow.io",
    "https://www.chitkara.edu.in",
    "https://www.youtube.com",
    "https://ingresoseguroitau.carrd.co",
    "http://update-your-amazon-account.com"
]

for url in test_urls:
    predict_url(url)


C:\Users\farus\AppData\Local\Temp\ipykernel_26184\1134771837.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["status"] = df["Label"].replace({"Legitimate": 0, "Phishing": 1})


[LightGBM] [Info] Number of positive: 140645, number of negative: 128754
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003448 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1080
[LightGBM] [Info] Number of data points in the train set: 269399, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.522069 -> initscore=0.088335
[LightGBM] [Info] Start training from score 0.088335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



🔍 URL: http://paypal-secure-login.com
🚀 LightGBM Prediction: Phishing (Confidence: 1.0000)
🚀 CatBoost Prediction: Phishing (Confidence: 0.9996)


D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



🔍 URL: https://www.chitkara.edu.in
🚀 LightGBM Prediction: Legitimate (Confidence: 0.0024)
🚀 CatBoost Prediction: Legitimate (Confidence: 0.0047)


D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



🔍 URL: https://www.youtube.com
🚀 LightGBM Prediction: Legitimate (Confidence: 0.0114)
🚀 CatBoost Prediction: Legitimate (Confidence: 0.0061)

🔍 URL: https://ingresoseguroitau.carrd.co
🚀 LightGBM Prediction: Phishing (Confidence: 0.9997)
🚀 CatBoost Prediction: Phishing (Confidence: 0.9889)

🔍 URL: http://update-your-amazon-account.com
🚀 LightGBM Prediction: Phishing (Confidence: 1.0000)
🚀 CatBoost Prediction: Phishing (Confidence: 0.9996)


D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\tensorflow\myvenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [5]:
import joblib
joblib.dump(lgb_model, "lightgbm_phishing_model.pkl")


['lightgbm_phishing_model.pkl']

In [7]:
cat_model.save_model("catboost_phishing_model.cbm", format="cbm")


In [8]:
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']